In [6]:
""""
Copyright (c) Meta Platforms, Inc. and affiliates.
All rights reserved.

This source code is licensed under the license found in the
LICENSE file in the root directory of this source tree.
"""

import numpy as np 
import pandas as pd 


In [7]:
def compute_minhi_mlwe(s, k, utotal, step=1):
    n = len(s)//k
    s = s.reshape((k,n, -1)).swapaxes(0,1)
    u = utotal//k
    hi = s[:u].sum(axis=0).sum(axis=0)
    minhi = hi

    for i in range(0, n, step):
        hi -= s[i].sum(axis=0)
        hi += s[(i+u)%n].sum(axis=0)
        minhi = np.minimum(minhi, hi)
 
    return minhi


def sample_s(n, h):
    s = np.zeros(n, dtype = int)
    ones = np.random.choice(n, size=h, replace=False)
    s[ones] = 1
    assert s.sum() == h
    return s

def sample_m_s(n, h, m):
    s = np.vstack([sample_s(n, h) for _ in range(m)]).T
    return s

def estimate_minh_probability(n, k, h, u, m):
    s = sample_m_s(n*k, h, m)
    samples = compute_minhi_mlwe(s, k, u)
    values, counts = np.unique(samples, return_counts=True)
    probs = np.zeros(h+1)
    probs[values] = counts/counts.sum()
    return probs

In [8]:
N = 1024
k = 1
n = N//k
u = 716
min_h = 5
max_h = 12
m = 200000


In [10]:
cumsums = dict()
for h in range(min_h, max_h + 1):
    probs = estimate_minh_probability(n, k, h, u, m)
    cumsums[h] = np.cumsum(probs[:5])

In [11]:
data = 100*pd.DataFrame.from_dict(cumsums)
data = data.round(2)
data.index = data.index.rename('Cruel Bits')
data

,5,6,7,8,9,10,11,12
Cruel Bits,,,,,,,,
0,4.05,1.39,0.52,0.16,0.06,0.02,0.01,0.00
1,33.64,15.46,6.58,2.59,1.03,0.38,0.13,0.05
2,94.57,62.30,34.48,16.99,7.75,3.31,1.33,0.55
3,100.00,99.92,87.98,57.72,32.77,16.81,8.03,3.70
4,100.00,100.00,100.00,98.54,80.34,52.12,30.25,16.01
